# Series Collection

Often users of the FRED api web service will want analyze multiple economic series. This can be done with `FredSeries` alone, but can be tedious and cumbersome. `pyfredapi` offers the `SeriesCollection` class to streamline the process of collecting and munging the data to get ready for plotting and analysis.

A `SeriesCollection` object is a set of `SeriesData` objects. `SeriesCollection` provide helpful methods to:

* List the metadata (frequency, seasonality, etc.) of the series in the collection
* Merge series dataframes into a long dataframe
* Merge series dataframes into a wide dataframe by index
* Merge series dataframes into a wide dataframe by date

In [ ]:
from pyfredapi import SeriesCollection
from pyfredapi.api.series import SeriesData
from rich import print as rprint

## Collect data with `.add_series()`

By default the column holding the series values will be renamed to the series id

In [ ]:
cpi_sc = SeriesCollection()
cpi_series = ["CPIAUCSL", "CPILFESL"]
cpi_sc.add_series(series_ids=cpi_series)

### Remove series with `.drop_series()`

In [ ]:
cpi_sc.drop_series("CPILFESL")

In [ ]:
cpi_sc.list_series()

## Accessing the data in the `SeriesCollection`

### Access via attribute

In [ ]:
type(cpi_sc.CPIAUCSL)

In [ ]:
rprint(cpi_sc.CPIAUCSL.info)

In [ ]:
cpi_sc.CPIAUCSL.data.head()

### Access via `data` dictionary

In [ ]:
cpi_sc.data["CPIAUCSL"].data.head()

## Rename series in the collection

### Rename on add

You can rename the series when adding them to the collection. Renaming can be done with a dictionary mapping the series id to the new name, or with a function which parses the series title into the new name.

In [ ]:
# Rename with a dictionary
new_names = {
    "CPIAUCSL": "cpi_all_items",
    "CPILFESL": "cpi_all_items_less_food_and_energy",
}

cpi_sc = SeriesCollection()
cpi_sc.add_series(
    series_ids=cpi_series,
    rename=new_names
)

In [ ]:
cpi_sc.CPIAUCSL.data.head()

In [ ]:
cpi_sc.CPILFESL.data.head()

### Rename after add

You can rename series in the collection with the `rename_series` method. Works the same way as renaming on add

In [ ]:
def parse_cpi_title(title: str) -> str:
    """Function to parse a CPI series title into a human readable label."""
    return (
        title.lower()
        .replace("consumer price index", "cpi ")
        .replace(" for all urban consumers: ", "")
        .replace(" in u.s. city average", "")
        .replace(" ", "_")
        #.capitalize()
    )

cpi_sc.rename_series(rename=parse_cpi_title)

In [ ]:
cpi_sc.CPIAUCSL.data

In [ ]:
cpi_sc.CPIAUCSL.data.head()

## List information about series in the collection

#### List series in the collection

In [ ]:
cpi_sc.list_series()

### List frequency, seasonality, and end data

In [ ]:
cpi_sc.list_frequency()

In [ ]:
cpi_sc.list_seasonality()

In [ ]:
cpi_sc.list_units()

In [ ]:
cpi_sc.list_end_date()

In [ ]:
cpi_sc.list_start_date()

## Merge data

`SeriesCollection` support merging into long and wide formats. By default the series ID will be used as the column name or observation label.

### Merge long

...

In [ ]:
cpi_long = cpi_sc.merge_long()
cpi_long

In [ ]:
set(cpi_long.series.tolist())

### Merge wide

...

In [ ]:
cpi_wide = cpi_sc.merge_wide()
cpi_wide.tail()

### Merge As-Of

...

In [ ]:
cpi_asof = cpi_sc.merge_asof(base_series_id="CPIAUCSL")
cpi_asof.tail()